In [1]:
import ete3
import pandas as pd
import numpy as np

In [2]:
t = ete3.Tree('ncbi.tre')

In [3]:
t.show()

In [4]:
taxids = pd.read_csv('allspecies.taxid', delimiter='\t', header=None)

In [5]:
taxids

,0,1
0,Ananas comosus,4615
1,Amaranthus hypochondriacus,28502
2,Asparagus officinalis,4686
3,Arabidopsis thaliana,3702
4,Amborella trichopoda,13333
5,Brachypodium distachyon,15368
6,Brassica oleracea,3712
7,Cicer arietinum,3827
8,Citrus clementina,85681
9,Capsella grandiflora,264402


In [6]:
name2species = pd.read_csv('name2species.csv', header=None)

In [7]:
merged = taxids.merge(name2species, left_on=0, right_on=1)

In [8]:
merged.head()

,key_0,0_x,1_x,0_y,1_y
0,Ananas comosus,Ananas comosus,4615,Acomosus,Ananas comosus
1,Amaranthus hypochondriacus,Amaranthus hypochondriacus,28502,Ahypochondriacus,Amaranthus hypochondriacus
2,Asparagus officinalis,Asparagus officinalis,4686,Aofficinalis,Asparagus officinalis
3,Arabidopsis thaliana,Arabidopsis thaliana,3702,Athaliana,Arabidopsis thaliana
4,Amborella trichopoda,Amborella trichopoda,13333,Atrichopoda,Amborella trichopoda


In [9]:
metadata = pd.read_csv('metaall.csv')

In [10]:
metadata.loc[:,['N', 'N50']]

,N,N50
0,0.0178,11759267
1,0.0001,24364990
2,0.0299,131339754
3,0.0016,23459830
4,0.0539,4927027
5,0.0016,59130575
6,0.0666,40895475
7,0.0930,39989001
8,0.0206,31410901
9,0.1100,112041


In [11]:
merged = merged.merge(metadata, left_on='0_y', right_on='species')

In [12]:
merged.loc[:, '#Names'] = merged.loc[:,['key_0','1_x']].apply(lambda x: ' - '.join([str(w) for w in x]), axis=1)

In [13]:
# todo select what one actually wants and scale
merged.loc[:,['#Names','N', 'N50']].to_csv('test.csv', sep='\t', index=False)

In [14]:
merged.loc[:,['#Names','N', 'N50']]

,#Names,N,N50
0,Ananas comosus - 4615,0.0178,11759267
1,Amaranthus hypochondriacus - 28502,0.0001,24364990
2,Asparagus officinalis - 4686,0.0299,131339754
3,Arabidopsis thaliana - 3702,0.0016,23459830
4,Amborella trichopoda - 13333,0.0539,4927027
5,Brachypodium distachyon - 15368,0.0016,59130575
6,Brassica oleracea - 3712,0.0666,40895475
7,Cicer arietinum - 3827,0.0930,39989001
8,Citrus clementina - 85681,0.0206,31410901
9,Capsella grandiflora - 264402,0.1100,112041


In [15]:
t = ete3.ClusterTree('ncbi.mod.tre', text_array='test.csv')
ts = ete3.TreeStyle()
ts.show_leaf_name = True

In [16]:
t.show("heatmap", tree_style=ts)